Data Cleaning

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:

# Load the three datasets
good_df = pd.read_csv("good.csv")
low_bad_df = pd.read_csv("low bad.csv")
high_bad_df = pd.read_csv("high bad.csv")

# Add a new column to identify the quality class
good_df['Quality'] = 'good'
low_bad_df['Quality'] = 'low_bad'
high_bad_df['Quality'] = 'high_bad'

# Combine all three into a single dataset
combined_df = pd.concat([good_df, low_bad_df, high_bad_df], ignore_index=True)

# Preview the combined dataset
print(combined_df.head())

In [ ]:
# Clean column names
combined_df.columns = (
    combined_df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

# Convert Set Time to datetime
combined_df['set_time'] = pd.to_datetime(combined_df['set_time'], dayfirst=True)


In [ ]:
# Preview all columns
print("All columns:\n", combined_df.columns.tolist())

# Create lists based on column naming patterns
sp_columns = [col for col in combined_df.columns if "_sp" in col]
pv_columns = [col for col in combined_df.columns if "_pv" in col]

# Metadata columns (manually listed)
meta_columns = ['vyp_batch', 'part', 'set_time', 'quality']

# Optional: Remaining system-specific or ungrouped columns
other_columns = list(set(combined_df.columns) - set(sp_columns) - set(pv_columns) - set(meta_columns))

# Print summaries
print(f"\nSet Point (SP) columns: {len(sp_columns)} →", sp_columns)
print(f"\nProcess Variable (PV) columns: {len(pv_columns)} →", pv_columns)
print(f"\nOther columns: {len(other_columns)} →", other_columns)


In [ ]:
for col in sp_columns + pv_columns:
    combined_df[f"{col}_lag1"] = combined_df[col].shift(1)


In [ ]:
for col in sp_columns + pv_columns:
    combined_df[f"{col}_roll3"] = combined_df[col].rolling(window=3).mean()


In [ ]:
for col in sp_columns + pv_columns:
    combined_df[f"{col}_delta"] = combined_df[col].diff()


In [ ]:
combined_df = pd.get_dummies(combined_df, columns=['part'])


In [ ]:
combined_df.dropna(inplace=True)


In [ ]:
combined_df.to_csv("combined_cleaned_features.csv", index=False)


In [13]:
# Rolling mean over 3 steps
for col in sp_columns + pv_columns:
    combined_df[f"{col}_roll3"] = combined_df[col].rolling(window=3).mean()

# Rate of change (delta)
for col in sp_columns + pv_columns:
    combined_df[f"{col}_delta"] = combined_df[col].diff()


In [19]:
# One-hot encode yeast type
#combined_df = pd.get_dummies(combined_df, columns=['part'])

# Drop nulls from lag/rolling/diff
combined_df.dropna(inplace=True)


In [20]:

# Define features and labels
exclude = ['vyp_batch', 'set_time', 'quality']
X = combined_df.drop(columns=exclude)
y = combined_df['quality']

# 70% training, 30% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

        good       0.97      0.98      0.97      3990
    high_bad       0.98      0.98      0.98      4349
     low_bad       0.96      0.91      0.94       773

    accuracy                           0.97      9112
   macro avg       0.97      0.96      0.96      9112
weighted avg       0.97      0.97      0.97      9112

[[3901   64   25]
 [  77 4267    5]
 [  53   13  707]]
